# Spline Model on Split by Sex Data

In [1]:
library("tidyverse")
library("dplyr")
library("mgcv")
library("mgcViz")

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.4     v dplyr   1.0.7
v tidyr   1.1.3     v stringr 1.4.0
v readr   2.0.1     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: nlme


Attaching package: 'nlme'


The following object is masked from 'package:dplyr':

    collapse


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Warning message:
"package 'mgcViz' was built under R version 4.1.2"
Loading required package: qgam

Warning message:
"package 'qgam' was built under R version 4.1.2"
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Registered S3 method overwritten by 'mgcViz':
  method from  
  +.gg   GGally


Attaching pac

In [5]:
dat = read_delim('violenceKNNResp_wSex_NotSplitImp.csv', delim = ",")
knn_dat = read_delim('violenceKNN.csv', delim = ",") %>% select(-c(violenceScore))
split_dat = right_join(dat, knn_dat, by = c("year", "sitename")) %>% 
    mutate(violenceScore = `violence score`) %>% 
    select(-c(4,5))

Rows: 262 Columns: 4

-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (1): sitename
dbl (3): year, sex, violence score


i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `` -> ...1

Rows: 122 Columns: 10

-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (1): sitename
dbl (9): ...1, year, violenceScore, AllAgesInPovertyPercent, UnderAge18InPov...


i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [9]:
head(split_dat)

year,sitename,sex,AllAgesInPovertyPercent,UnderAge18InPovertyPercent,MedianHouseholdIncomeInDollars,UnemploymentRate,Population,SNAP,violenceScore
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2003,"Borough of Bronx, NY (NYG)",1,26.9,37.4,27855,9.9,1362373,97.29,36.33420
2003,"Borough of Bronx, NY (NYG)",2,26.9,37.4,27855,9.9,1362373,97.29,64.92268
2003,"Borough of Brooklyn, NY (NYH)",1,23.4,33.3,30672,8.1,2472999,97.29,26.94144
2003,"Borough of Brooklyn, NY (NYH)",2,23.4,33.3,30672,8.1,2472999,97.29,69.22551
2003,"Borough of Manhattan, NY (NYI)",1,18.7,30.9,43930,6.6,1562154,97.29,26.21359
2003,"Borough of Manhattan, NY (NYI)",2,18.7,30.9,43930,6.6,1562154,97.29,57.53186


## Fitting with spline

In [16]:
both_smooth = gam(violenceScore ~ 
              s(SNAP, by = sex, bs = "cr", k = 5) + 
              s(UnemploymentRate, by = sex, bs = "cr", k = 5) + 
              s(as.factor(split_dat$sitename), bs = "re"), data = split_dat)
both_AIC = AIC(both_smooth)

UnemployRate_smooth = gam(violenceScore ~ SNAP +
              s(UnemploymentRate, by = sex, bs = "cr", k = 5) + 
              s(as.factor(split_dat$sitename), bs = "re"), data = split_dat)
UnemployRate_AIC = AIC(UnemployRate_smooth)

SNAP_smooth = gam(violenceScore ~ 
              s(SNAP, by = sex, bs = "cr", k = 5) + 
              UnemploymentRate + s(as.factor(split_dat$sitename), bs = "re"), data = split_dat)
SNAP_AIC = AIC(SNAP_smooth)

neither_smooth = gam(violenceScore ~ SNAP + UnemploymentRate +
              s(as.factor(split_dat$sitename), bs = "re"), data = split_dat)
neither_AIC = AIC(neither_smooth)

AIC_mat = matrix(c(both_AIC, SNAP_AIC, UnemployRate_AIC, neither_AIC), nrow = 4)

rowNames = c("Both Smooth AIC", "SNAP Smooth AIC", "Unemployment Rate Smooth AIC", "Neither Smooth AIC")
row.names(AIC_mat) = rowNames

AIC_mat

Both Smooth AIC,1565.375
SNAP Smooth AIC,1636.591
Unemployment Rate Smooth AIC,1582.434
Neither Smooth AIC,2015.223


### Calculating MSE

In [42]:
# Both fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ s(SNAP, by = sex, bs = "cr", k = k) + 
                                  s(UnemploymentRate, by = sex, bs = "cr", k = k) + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])
          
    
  }#for
  
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

both_MSE = CrossValMSE(k = 5, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)

$mse
[1] 117.1162

$mse_male
[1] 166.868

$mse_female
[1] 67.36445

In [23]:
# UnemploymentRate fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ SNAP + 
                                  s(UnemploymentRate, by = sex, bs = "cr", k = k) + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])
          
    
  }#for
  
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

UnemployRate_MSE = CrossValMSE(k = 5, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)

In [24]:
# SNAP fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ s(SNAP, by = sex, bs = "cr", k = k) + 
                                  UnemploymentRate + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])
          
    
  }#for
  
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

SNAP_MSE = CrossValMSE(k = 5, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)

In [25]:
# Neither fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ SNAP + UnemploymentRate + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])
          
    
  }#for
  
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

neither_MSE = CrossValMSE(k = 5, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)

In [34]:
MSE_mat = matrix(c(both_MSE, SNAP_MSE, UnemployRate_MSE, neither_MSE), nrow = 4, ncol = 3, byrow = TRUE)

rowNames = c("Both Smooth MSE", "SNAP Smooth MSE", "Unemployment Rate Smooth MSE", "Neither Smooth MSE")
rownames(MSE_mat) = rowNames

colNames = c("Total", "Male", "Female")
colnames(MSE_mat) = colNames

MSE_mat

Total    Male     Female  
Both Smooth MSE              45.45004 58.7142  32.18588
SNAP Smooth MSE              45.92002 59.3765  32.46353
Unemployment Rate Smooth MSE 57.79627 73.01279 42.57976
Neither Smooth MSE           221.7447 236.421  207.0683

## Find Best Number of Knots

### Same Number of Knots for Both SNAP and UnemploymentRate

In [50]:
# Both fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ s(SNAP, by = sex, bs = "cr", k = k) + 
                                  s(UnemploymentRate, by = sex, bs = "cr", k = k) + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])

  }#for
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

MSE = data.frame(matrix(0, nrow = 25, ncol = 3))
names = c("Total MSE", "Male MSE", "Female MSE")
rows = c(1:25)
colnames(MSE) = names
rownames(MSE) = rows


for (i in (1:25)) {
    MSE[i,] = CrossValMSE(k = i, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)
}

In [55]:
which.min(MSE[,1])
which.min(MSE[,2])
which.min(MSE[,3])

MSE[which.min(MSE[,1]),]

MSE

[1] 17

[1] 17

[1] 17

,Total MSE,Male MSE,Female MSE
,<dbl>,<dbl>,<dbl>
17,37.91952,47.41375,28.42528


,Total MSE,Male MSE,Female MSE
,<dbl>,<dbl>,<dbl>
1,50.88666,65.71165,36.06167
2,50.88666,65.71165,36.06167
3,50.88666,65.71165,36.06167
4,45.52147,58.67890,32.36404
5,45.45004,58.71420,32.18588
6,42.37099,54.16952,30.57247
7,42.34733,53.86236,30.83231
8,42.49501,54.30429,30.68574
9,42.51254,53.95032,31.07475


### Different Number of Knots for SNAP and UnemploymentRate

In [74]:
# Both fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, m, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ s(SNAP, by = sex, bs = "cr", k = k) + 
                                  s(UnemploymentRate, by = sex, bs = "cr", k = m) + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])

  }#for
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

MSE = data.frame(matrix(0, nrow = 400, ncol = 3))
names = c("Total MSE", "Male MSE", "Female MSE")
rows = c(1:400)
colnames(MSE) = names
rownames(MSE) = rows


for (i in (4:23)) {
    for (j in 4:23)
        MSE[(j-3) + (20*(i-4)),] = CrossValMSE(k = i, m = j, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)
}

In [77]:
which.min(MSE[,1])
which.min(MSE[,2])
which.min(MSE[,3])

MSE[which.min(MSE[,1]), ]

# SNAP(k = 17), UnemploymentRate(k = 6)

[1] 263

[1] 321

[1] 263

,Total MSE,Male MSE,Female MSE
,<dbl>,<dbl>,<dbl>
263,36.68794,45.73309,27.64278


,Total MSE,Male MSE,Female MSE
,<dbl>,<dbl>,<dbl>
1,45.52147,58.67890,32.36404
2,45.33508,58.28624,32.38393
3,45.84442,59.28460,32.40425
4,45.77858,59.00807,32.54910
5,45.73707,58.96248,32.51165
6,45.80145,59.04297,32.55994
7,45.74495,58.96406,32.52584
8,45.67712,58.85649,32.49775
9,45.68025,58.86176,32.49875


In [84]:
# Both fit with smoothing spline
library("gtools")
options(warn = -1)

CrossValMSE = function(k, m, modelGroup, nLeftOut, dat, mseSplitOn) {
  
  # Make all combinations for pairs of county
  leftOutCols = combinations(length(t(unique(dat[modelGroup]))), nLeftOut, v = t(unique(dat[modelGroup])))
  
  msesSplit = rep(0,dim(leftOutCols)[1])
  msesSplit_male = rep(0,dim(leftOutCols)[1])
  msesSplit_female = rep(0,dim(leftOutCols)[1])
  
  for (lo in 1:dim(leftOutCols)[1]) {
    # Create training and testing set
    # Training set includes all but two counties
    dat_keep = dat %>% filter(sitename != leftOutCols[lo,1] & sitename != leftOutCols[lo,2])
    dat_keep$sitename = as.factor(dat_keep$sitename)
    
    dat_remove = dat %>% filter(sitename == leftOutCols[lo,1] | sitename == leftOutCols[lo,2])
    dat_remove$sitename = as.factor(dat_remove$sitename)
    
    # Fit the spline model
    gam_mod = gam(violenceScore ~ s(SNAP, by = sex, bs = "cr", k = k) + 
                                  s(UnemploymentRate, by = sex, bs = "cr", k = m) + 
                                  s(sitename, bs = "re"), data = dat_keep)

    pred = predict(gam_mod, dat_remove, exclude = "s(sitename)")
    
    # Calculate the total mse
    msesSplit[lo] = sum((dat_remove$violenceScore - pred)^2) / length(pred)
    
    male_index = which(dat_remove$sex == 2)
    female_index = which(dat_remove$sex == 1)
    
    # Calculate the mse for male and for female
    msesSplit_male[lo] = sum((dat_remove[male_index, "violenceScore"] - pred[male_index])^2) / length(pred[male_index])
    msesSplit_female[lo] = sum((dat_remove[female_index, "violenceScore"] - pred[female_index])^2) / length(pred[female_index])

  }#for
  
  return(list(mse = mean(msesSplit), mse_male = mean(msesSplit_male), mse_female = mean(msesSplit_female)))
  
}#function

MSE_best = CrossValMSE(k = 17, m = 6, modelGroup = 2, nLeftOut = 2, dat = split_dat, mseSplitOn = 3)
MSE_best


$mse
[1] 36.68794

$mse_male
[1] 45.73309

$mse_female
[1] 27.64278